In [1]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from bs4 import BeautifulSoup
import requests

ModuleNotFoundError: No module named 'torch'

In [ ]:
def normalizza(val, vmin, vmax):
    if vmax == vmin:
        return 0
    return (val - vmin) / (vmax - vmin)

def convert_numeric_columns(df, exclude=[]):
    for col in df.columns:
        if col not in exclude:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df.fillna(0)

def prepare_tensors_for_day(df_features, df_votes, giornata, feat_cols):
    df_day = df_features[df_features['Giornata'] == giornata].copy()
    if df_day.empty:
        return None
    votes_day = df_votes[df_votes['Giornata'] == giornata][['Player','Voto']]
    df_day = df_day.merge(votes_day, on='Player', how='left')
    df_day['Voto'] = df_day['Voto'].fillna(0.0)
    players = df_day['Player'].tolist()
    pos = df_day['Pos'].tolist()
    X = torch.tensor(df_day[feat_cols].values, dtype=torch.float32)
    y = torch.tensor(df_day['Voto'].values, dtype=torch.float32)
    X = (X - X.mean(0, keepdim=True)) / (X.std(0, keepdim=True) + 1e-6)
    pos_idx = {}
    ROLE_SLOTS = {'GK': 1, 'DF': 3, 'MF': 4, 'FW': 3}
    for r in ROLE_SLOTS.keys():
        pos_idx[r] = torch.tensor([i for i,p in enumerate(pos) if p.startswith(r)], dtype=torch.long)
    return X, y, pos_idx, players

In [ ]:
class LinearScorer(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.w = nn.Parameter(torch.randn(n_features) * 0.01)
        self.b = nn.Parameter(torch.tensor(0.0))
    def forward(self, X):
        return X.matmul(self.w) + self.b

In [ ]:

def expected_formation_vote(logits, y, pos_idx, role_slots, temp=0.1):
    device = logits.device
    total_expected = torch.tensor(0.0, device=device)
    for role, k in role_slots.items():
        idx = pos_idx.get(role)
        if idx is None or idx.numel() == 0:
            continue
        logits_role = logits[idx]
        probs = torch.softmax(logits_role / temp, dim=0)
        p_scaled = probs * k
        expected_role_vote = torch.sum(p_scaled * y[idx])
        total_expected = total_expected + expected_role_vote
    return total_expected

In [ ]:


def train_weights(df_features, df_votes, feat_cols, train_giornate,
                  role_slots={'GK':1,'DF':3,'MF':4,'FW':3},
                  n_epochs=200, lr=0.05, temp=0.1, weight_decay=1e-4):
    n_features = len(feat_cols)
    model = LinearScorer(n_features)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    for epoch in range(n_epochs):
        epoch_loss = 0.0
        count = 0
        for g in train_giornate:
            prepared = prepare_tensors_for_day(df_features, df_votes, g, feat_cols)
            if prepared is None:
                continue
            X, y, pos_idx, players = prepared
            logits = model(X)
            expected_vote = expected_formation_vote(logits, y, pos_idx, role_slots, temp=temp)
            loss = - expected_vote
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            count += 1
        if (epoch+1) % 20 == 0 or epoch==0:
            print(f"Epoch {epoch+1}/{n_epochs} avg loss {epoch_loss/max(1,count):.4f}")
    return model

In [ ]:
def scrape_pianetafanta():
    url = 'https://www.pianetafanta.it/voti-ufficiosi.asp'
    r = requests.get(url, timeout=15)
    tables = pd.read_html(r.text)
    # Qui puoi filtrare la tabella giusta manualmente o con indice
    df_votes = tables[0]  # esempio, modificare se necessario
    # Aggiungi colonna 'Giornata' se mancante
    df_votes['Giornata'] = 1  # esempio placeholder
    df_votes.rename(columns={'Nome':'Player','Voto':'Voto'}, inplace=True)
    return df_votes


In [ ]:
# Cell 7: Esempio di uso
# feat_cols = ['Titolarità','Forma','BonusPot','Affidabilità','Calendario','Penalità']
# train_giornate = sorted(df_features['Giornata'].unique())[:20]
# model = train_weights(df_features, df_votes, feat_cols, train_giornate)